# Data Processing

----
Credits for the bulk of the code to *Yaron Blinder, 'Predicting 30-day ICU readmissions from the MIMIC-III database'* <br>
https://github.com/YaronBlinder/MIMIC-III_readmission <br>

And from: https://github.com/bvanaken/clinical-outcome-prediction/blob/master/tasks/mimic_utils.py

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os
import psycopg2
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# PARAMETERS

icu_stays = True # set to TRUE if we want to have only ICU stays
combine_dup = True # True if we want to combine the duplicates
remove_dup = True # True if we later want to remove the shortest note
seq_prio = False # set to True if we want to consider only the top priority code of hospitalization

expanded_def = True # set to True if we want to consider future readmissions and avoid using CMS 

if icu_stays == True:
    icu_folder = 'icu_only'
    if expanded_def:
        icu_folder = 'expanded'
else:
    icu_folder = 'all_hosp'

In [3]:
path_to_repo = os.path.dirname(os.getcwd())

print(path_to_repo)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission


In [4]:
path_to_data = os.path.join(path_to_repo,"data", icu_folder,"")
os.makedirs(path_to_data, exist_ok=True) # we create the directory if it does not exist
print(path_to_data)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\data\expanded\


In [5]:
# create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
ps_password = 'postgres'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser, password = ps_password)
cur = con.cursor()

In [6]:
# SQL query (gets all_data view defined elsewhere)
query = \
"""
select * from note_filter
"""

In [7]:
# Perform SQL query
cur.execute('SET search_path to ' + schema_name)
df = pd.read_sql_query(query,con)

In [8]:
if icu_stays == True:
    print("Observations before filtering:{}".format(df.shape))
    df.dropna(axis = 0, subset=["first_careunit","last_careunit"], inplace = True)
    print("Observations after filtering:{}".format(df.shape))

Observations before filtering:(59786, 14)
Observations after filtering:(58824, 14)


Eliminate all errors within discharge notes.

In [9]:
df = df[df.iserror != None]
# and drop the column since it won't be needed
df.drop('iserror', axis = 1, inplace = True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58824 entries, 0 to 59785
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_id      58824 non-null  int64         
 1   hadm_id         58824 non-null  int64         
 2   admittime       58824 non-null  datetime64[ns]
 3   dischtime       58824 non-null  datetime64[ns]
 4   deathtime       6387 non-null   datetime64[ns]
 5   first_careunit  58824 non-null  object        
 6   last_careunit   58824 non-null  object        
 7   age             58824 non-null  float64       
 8   gender          58824 non-null  object        
 9   marital_status  56124 non-null  object        
 10  insurance       58824 non-null  object        
 11  diagnosis       58823 non-null  object        
 12  text            58824 non-null  object        
dtypes: datetime64[ns](3), float64(1), int64(2), object(7)
memory usage: 6.3+ MB


We want to delete all the patients who died either during their first readmission or in later readmissions.

In [11]:
df_clean = df[np.isnat(df['deathtime'])].copy()

In [12]:
# sort our values by subject id
df_clean.sort_values('subject_id', inplace = True)
# drop deathtime column since we won't need it anymore
df_clean.drop('deathtime', axis = 1, inplace = True)

In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52437 entries, 47209 to 44853
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_id      52437 non-null  int64         
 1   hadm_id         52437 non-null  int64         
 2   admittime       52437 non-null  datetime64[ns]
 3   dischtime       52437 non-null  datetime64[ns]
 4   first_careunit  52437 non-null  object        
 5   last_careunit   52437 non-null  object        
 6   age             52437 non-null  float64       
 7   gender          52437 non-null  object        
 8   marital_status  50308 non-null  object        
 9   insurance       52437 non-null  object        
 10  diagnosis       52436 non-null  object        
 11  text            52437 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(7)
memory usage: 5.2+ MB


In [14]:
df_clean.reset_index(inplace = True, drop = True)

In [15]:
df_clean.shape

(52437, 12)

**ISSUE**: we have some duplicates.

In [16]:
# remove duplicates and keep the later ones
df_clean = df_clean.sort_values(by=["subject_id", "hadm_id", 'admittime'])
df_clean = df_clean.drop_duplicates(subset=["text"], keep="last")
df_clean.reset_index(inplace = True, drop = True)

In [17]:
df_clean.shape

(48765, 12)

Not only duplicates, we have duplicates in terms of subject_id, hamd_id couples.

In [18]:
df_clean['duplicate'] = df_clean.duplicated(['subject_id','hadm_id'], keep = False)

In [19]:
df_clean.loc[4, 'text']

'Name:  [**Known lastname 9900**], [**Known firstname **] C                     Unit No:  [**Numeric Identifier 9993**]\n\nAdmission Date:  [**2167-1-8**]     Discharge Date:  [**2167-1-15**]\n\nDate of Birth:   [**2127-2-27**]     Sex:  F\n\nService:  Cardiac surgery\n\nIf the previously right coronary artery stenosis is a cause\nfor concern or patient morbidity in the future, this lesion\ncan certainly be addressed percutaneously with balloon\nangiography. She will be followed very closely by Dr. [**Last Name (STitle) 677**]\nas an outpatient, and if this lesion does cause her problems,\nthis may be addressed.\n\n\n                          [**Doctor Last Name **] [**Last Name (Prefixes) **], M.D.  [**MD Number(1) 681**]\n\nDictated By:[**Last Name (NamePattern1) 4660**]\n\nMEDQUIST36\n\nD:  [**2167-1-14**]  16:39\nT:  [**2167-1-14**]  18:51\nJOB#:  [**Job Number 9994**]\n'

In [20]:
df_clean.loc[5, 'text']

'Admission Date:  [**2167-1-8**]       Discharge Date:  [**2167-1-15**]\n\nDate of Birth:   [**2127-2-27**]       Sex:  F\n\nService:  Cardiac surgery\n\nCHIEF COMPLAINT:  Chest pain.\n\nHISTORY OF PRESENT ILLNESS:  This is a 39-year-old woman with\ndiabetes, hypertension, hyperlipidemia and obesity, with a\none to two months of chest burning with exertion. For the\npast six months, she has been participating in a new vigorous\nexercise program to lose weight. Her symptoms do gradually\nresolve with rest, but they have started to occur now with\nwalking. She does acknowledge that there is associated\nnausea, diaphoresis and shortness of breath. Now recently she\nstarted to get symptoms for the past two days while at rest.\nShe was referred for an outpatient exercise tolerance test,\nwhere she had chest pain and significant EKG changes. She was\nreferred to [**Hospital1 18**] for cardiac catheterization today, which\nrevealed significant left main artery disease. Just prior to\nher tran

**Option 1:** We remove the shortest note, assuming that its information will be contained in the longer discharge note. <br>
**Option 2:** We merge the two notes, in case they are important addendums.

We have 37925 unique admission/subject IDs, and we have keys going up to 7 duplicates.

In [21]:
df_clean['key'] = df_clean.subject_id.astype(str) + '/' + df_clean.hadm_id.astype(str)

In [22]:
np.unique(df_clean.key.value_counts().values, return_counts = True)

(array([1, 2, 3, 4, 5, 6, 7], dtype=int64),
 array([37925,  4151,   605,   130,    26,    11,     1], dtype=int64))

In [23]:
df_clean.duplicated(['subject_id','hadm_id'], keep = False).value_counts()

False    37925
True     10840
dtype: int64

In [24]:
def merge_and_remove_dup(df_clean, combine_dup = True, remove_dup = True):
    """ Function to combine and remove duplicates 
    combine_dup: set to True if we want to combine duplicates text
    remove_dup: set to True if we want then to remove duplicates    
    """
    df_clean = df_clean.sort_values(by=["subject_id", "hadm_id", 'admittime'])
    df_clean['duplicate'] = df_clean.duplicated(['subject_id','hadm_id'], keep = False)
    # CODE TO REMOVE DUPLICATES -> here we are checking what note is the longes
    if combine_dup == False:
        for idx in np.arange(1,df_clean.shape[0]):
            if df_clean.duplicate[idx] == df_clean.duplicate[idx - 1] and df_clean.subject_id[idx] == df_clean.subject_id[idx - 1] and df_clean.hadm_id[idx] == df_clean.hadm_id[idx - 1]:     
                prev_note = len(df_clean.text[idx -1])
                current_note = len(df_clean.text[idx])
                max_lenght = max(prev_note, current_note)
                df_clean.loc[idx, 'duplicate'] = current_note != max_lenght
                df_clean.loc[idx-1, 'duplicate'] = prev_note != max_lenght

    # CODE TO COMBINE DUPLICATES -> here we are combining the two notes
    if combine_dup == True:
        for idx in np.arange(1,df_clean.shape[0]):
            if df_clean.duplicate[idx] == df_clean.duplicate[idx - 1] and df_clean.subject_id[idx] == df_clean.subject_id[idx - 1] and df_clean.hadm_id[idx] == df_clean.hadm_id[idx - 1]:     
                prev_note = len(df_clean.text[idx -1])
                current_note = len(df_clean.text[idx])
                max_lenght = max(prev_note, current_note)
                df_clean.loc[idx, 'duplicate'] = current_note != max_lenght
                df_clean.loc[idx-1, 'duplicate'] = prev_note != max_lenght
                if current_note != max_lenght:
                    df_clean.loc[idx -1, 'text'] = df_clean.text[idx -1] + '\n\n' + df_clean.text[idx]
                elif current_note == max_lenght:
                    df_clean.loc[idx, 'text'] = df_clean.text[idx] + '\n\n' + df_clean.text[idx-1]

    # CODE TO REMOVE DUPLICATES -> here we are removing the shortest notes (need to run this code in both options)
    if remove_dup == True:
        # get the number of actual duplicates
        print('Initial observations: {}'.format(df_clean.shape[0]))
        print(df_clean.duplicate.value_counts())
        # remove the duplicate rows
        df_clean = df_clean.loc[df_clean['duplicate'] == False,:]
        # sort our values by subject id
        df_clean.sort_values('subject_id', inplace = True)
        # drop duplicate column since we won't need it anymore
        df_clean.drop('duplicate', axis = 1, inplace = True)
        # and reset the index
        df_clean.reset_index(inplace = True, drop = True)
        print('Final observations: {}'.format(df_clean.shape[0]))
    return df_clean

In [25]:
df_clean = merge_and_remove_dup(df_clean, combine_dup = combine_dup, remove_dup = remove_dup)

Initial observations: 48765
False    43413
True      5352
Name: duplicate, dtype: int64
Final observations: 43413


There are still some rare instances in which there were was more than a single duplicate, we will thus repeat the process above to fix them.

In [26]:
df_clean.duplicated(['subject_id','hadm_id'], keep = False).value_counts()

False    42361
True      1052
dtype: int64

In [27]:
df_clean[df_clean.duplicated(['subject_id','hadm_id'], keep = False)].head(20)

,subject_id,hadm_id,admittime,dischtime,first_careunit,last_careunit,age,gender,marital_status,insurance,diagnosis,text,key
203,214,197273,2188-10-06 23:56:00,2188-11-04 11:40:00,CSRU,CSRU,63.442860,M,MARRIED,Private,"HYPOXIA,ACUTE CORONARY SYNDROME,HYPOTENSION,AC...",Admission Date: [**2188-10-6**] Dischar...,214/197273
204,214,197273,2188-10-06 23:56:00,2188-11-04 11:40:00,CCU,CCU,63.442860,M,MARRIED,Private,"HYPOXIA,ACUTE CORONARY SYNDROME,HYPOTENSION,AC...","Name: [**Known lastname 9454**], [**Known fir...",214/197273
216,228,167764,2132-10-30 02:16:00,2132-11-10 17:30:00,CCU,CCU,79.413360,F,WIDOWED,Medicare,LT FEMUR FRACTURE,Admission Date: [**2132-10-30**] Discha...,228/167764
217,228,167764,2132-10-30 02:16:00,2132-11-10 17:30:00,CCU,CCU,79.413360,F,WIDOWED,Medicare,LT FEMUR FRACTURE,"Name: [**Known lastname 9250**], [**Known fir...",228/167764
218,231,198608,2176-05-14 21:12:00,2176-06-05 15:00:00,MICU,MICU,36.731491,M,SINGLE,Private,PNEUMONIA,Admission Date: [**2176-5-14**] Dischar...,231/198608
219,231,198608,2176-05-14 21:12:00,2176-06-05 15:00:00,MICU,MICU,36.731491,M,SINGLE,Private,PNEUMONIA,Admission Date: [**2176-5-14**] Dischar...,231/198608
221,231,198608,2176-05-14 21:12:00,2176-06-05 15:00:00,MICU,MICU,36.731491,M,SINGLE,Private,PNEUMONIA,Admission Date: [**2176-5-14**] Dischar...,231/198608
311,330,197569,2133-02-27 12:30:00,2133-03-24 17:00:00,SICU,SICU,67.718173,M,SINGLE,Private,LEFT CAROTID STENOSIS/SDA,Admission Date: [**2133-2-27**] Dischar...,330/197569
312,330,197569,2133-02-27 12:30:00,2133-03-24 17:00:00,SICU,SICU,67.718173,M,SINGLE,Private,LEFT CAROTID STENOSIS/SDA,"Name: [**Known lastname 5057**],[**Known firs...",330/197569
343,362,142749,2112-07-10 02:31:00,2112-07-16 13:45:00,MICU,MICU,85.795459,F,WIDOWED,Medicare,HYPONATREMIA-R/O MYOCARDIAL INFARCTION-RHABDOM...,Admission Date: [**2112-7-10**] Dischar...,362/142749


In [28]:
for i in range(1,5):
    print('Iteration:{}'.format(i))
    df_clean = merge_and_remove_dup(df_clean, combine_dup = combine_dup, remove_dup = remove_dup)

Iteration:1
Initial observations: 43413
False    42872
True       541
Name: duplicate, dtype: int64
Final observations: 42872
Iteration:2
Initial observations: 42872
False    42827
True        45
Name: duplicate, dtype: int64
Final observations: 42827
Iteration:3
Initial observations: 42827
False    42825
True         2
Name: duplicate, dtype: int64
Final observations: 42825
Iteration:4
Initial observations: 42825
False    42825
Name: duplicate, dtype: int64
Final observations: 42825


In [29]:
df_clean.shape

(42825, 13)

In [30]:
df_clean.duplicated(['subject_id','hadm_id'], keep = False).value_counts()

False    42825
dtype: int64

In [31]:
np.unique(df_clean.key.value_counts().values, return_counts = True)

(array([1], dtype=int64), array([42825], dtype=int64))

In [32]:
df_clean.drop('key', axis = 1, inplace = True)

## Planned vs Unplanned Readmissions

We need to distinguish between planned and unplanned readmissions. In order to do so, we will employ the CMS Plannned Readmission Algorithm Version 3.0 (2015 update). Later version were ignored since they switched to ICD-10 codes.

![Planned Readmission Algorithm](img/planned_readmission.png)

First we will **load the icd codes for both diagnoses and procedures**.

In [33]:
# SQL query (gets all_data view defined elsewhere)
query_diag = \
"""
select * from icd_diag_data
"""

# SQL query (gets all_data view defined elsewhere)
query_proc = \
"""
select * from icd_proc_data
"""

In [34]:
# Perform SQL query
cur.execute('SET search_path to ' + schema_name)
diag_icd = pd.read_sql_query(query_diag,con)
proc_icd = pd.read_sql_query(query_proc,con)

In [35]:
proc_icd.shape

(240095, 6)

In [36]:
diag_icd.shape

(651047, 6)

Then we will upload the ICD-9 to CCS conversion tables, since the CMS algorithm uses a combination of ICD-9 codes and general CCS categories.

> Elixhauser A, Steiner C, Palmer L. Clinical Classifications Software (CCS), 2015. U.S. Agency for Healthcare Research and Quality. Available: http://www.hcup-us.ahrq.gov/toolssoftware/ccs/ccs.jsp

In [37]:
path_to_css = os.path.join(path_to_repo, "Readmission","css_categories","").replace("\\","/")
path_to_ccs_diag = f'{path_to_css}$dxref 2015.csv'
print(path_to_ccs_diag)

C:/Users/luca9/Documents/MIMIC-III Text Mining/mimim_iii_readmission/Readmission/css_categories/$dxref 2015.csv


In [38]:
ccs_diag = pd.read_csv(r"C:\Users\luca9\Documents\MIMIC-III Text Mining\Readmission\ccs_categories\$dxref 2015.csv", sep = ";")

ccs_proc = pd.read_csv(r"C:\Users\luca9\Documents\MIMIC-III Text Mining\Readmission\ccs_categories\$prref 2015.csv", sep = ";")

And drop the duplicates.

In [39]:
print("CCS-Diag: ",ccs_diag['ICD-9-CM CODE'].duplicated().sum())
print("CCS-Proc: ",ccs_proc['ICD-9-CM CODE'].duplicated().sum())

CCS-Diag:  384
CCS-Proc:  85


In [40]:
ccs_diag.drop_duplicates(subset = 'ICD-9-CM CODE', inplace = True)
ccs_proc.drop_duplicates(subset = 'ICD-9-CM CODE', inplace = True)

Then we perform a left-join of the MIMIC-III tables with the CCS ones to fetch the corresponding CCS code.

In [41]:
diag_icd = pd.merge(diag_icd, ccs_diag, how = 'left', left_on='icd9_code', right_on='ICD-9-CM CODE')
proc_icd = pd.merge(proc_icd, ccs_proc, how = 'left', left_on='icd9_code', right_on='ICD-9-CM CODE')

Then we import the categories from the Planned Readmission Algorithm.

In [42]:
path_to_excel = os.path.join(path_to_repo,"cms_planned_readmission","")
print(path_to_excel)

C:\Users\luca9\Documents\MIMIC-III Text Mining\mimim_iii_readmission\cms_planned_readmission\


In [43]:
xls = pd.ExcelFile(f'{path_to_excel}planned_readmission_algorithm.xlsx')

In [44]:
step_1 = pd.read_excel(xls, '1')
step_2 = pd.read_excel(xls, '2')
step_3 = pd.read_excel(xls, '3')
step_4 = pd.read_excel(xls, '4')

We first check how many procedures can be considered as a Planned Readmission (step 1).

In [45]:
step_1

,Type,Codification,Code,Description
0,Procedure,CCS,64,Bone marrow transplant
1,Procedure,CCS,105,Kidney transplant
2,Procedure,CCS,134,Cesarean section (Included only in all-payer p...
3,Procedure,CCS,135,Forceps; vacuum; and breech delivery (Included...
4,Procedure,CCS,176,Other organ transplantation


In [46]:
proc_icd['Step 1'] = proc_icd['CCS CATEGORY'].isin(list(step_1.Code))

In [47]:
proc_icd['Step 1'].value_counts()

False    239551
True        544
Name: Step 1, dtype: int64

We first check how many procedures can be considered as a Planned Readmission (step 2).

In [48]:
step_2

,Type,Codification,Code,Description
0,Diagnosis,CCS,45,Maintenance chemotherapy
1,Diagnosis,CCS,194,Forceps delivery (Included only in all-payer p...
2,Diagnosis,CCS,196,Normal pregnancy and/or delivery (Included onl...
3,Diagnosis,CCS,254,"Rehabilitation (Includes only V52.0, V52.1, V5..."


In [49]:
diag_icd['Step 2'] = diag_icd['CCS CATEGORY'].isin(list(step_2.Code))

In [50]:
diag_icd['Step 2'].value_counts()

False    650973
True         74
Name: Step 2, dtype: int64

Then check if the hospitalization includes a potentially planned procedure (step 3).

In [51]:
step_3

,Type,Codification,Code,Description
0,Procedure,CCS,3,Laminectomy; excision intervertebral disc
1,Procedure,CCS,5,Insertion of catheter or spinal stimulator and...
2,Procedure,CCS,9,Other OR therapeutic nervous system procedures
3,Procedure,CCS,10,Thyroidectomy; partial or complete
4,Procedure,CCS,12,Other therapeutic endocrine procedures
...,...,...,...,...
61,Procedure,ICD-9,3818,Endarterectomy leg vessel (from Procedure CCS ...
62,Procedure,ICD-9,5503,Percutaneous nephrostomy with and without frag...
63,Procedure,ICD-9,5504,Percutaneous nephrostomy with and without frag...
64,Procedure,ICD-9,9426,Electroshock therapy (from Procedure CCS 218-P...


In [52]:
# we first check by CCS codes
proc_icd['Step 3'] = proc_icd['CCS CATEGORY'].isin(list(step_3.loc[step_3.Codification == "CCS",].Code))

In [53]:
proc_icd['Step 3'].value_counts()

False    189094
True      51001
Name: Step 3, dtype: int64

In [54]:
# and then for ICD-9 codes
proc_icd['Step 3-ICD'] = proc_icd['icd9_code'].isin(list(step_3.loc[step_3.Codification != "CCS",].Code))

In [55]:
proc_icd['Step 3-ICD'].value_counts()

False    240095
Name: Step 3-ICD, dtype: int64

In [56]:
# we then get the maximum between the two rows - i.e. if any of the two is True, then it could be a planned readmission
proc_icd["Step 3 - def"] = proc_icd[["Step 3", "Step 3-ICD"]].max(axis=1)

And then check the Step 4 diagnoses.

In [57]:
# we first check by CCS codes
diag_icd['Step 4'] = diag_icd['CCS CATEGORY'].isin(list(step_4.loc[step_4.Codification == "CCS",].Code))

In [58]:
diag_icd['Step 4'].value_counts()

False    404892
True     246155
Name: Step 4, dtype: int64

In [59]:
# and then for ICD-9 codes
diag_icd['Step 4-ICD'] = diag_icd['icd9_code'].isin(list(step_4.loc[step_4.Codification != "CCS",].Code))

In [60]:
diag_icd['Step 4-ICD'].value_counts()

False    651047
Name: Step 4-ICD, dtype: int64

In [61]:
# we then get the maximum between the two rows - i.e. if any of the two is True, then it could be an unplanned readmission
diag_icd["Step 4 - def"] = diag_icd[["Step 4", "Step 4-ICD"]].max(axis=1)

**Option 2:** Consider only the priority code for the hospitalization (*seq_num = 1.0*).

In [62]:
if seq_prio == True:
    diag_icd = diag_icd[diag_icd.seq_num == 1]
    proc_icd = proc_icd[proc_icd.seq_num == 1]

Finally we get the unique values out of the procedures and diagnosis. If one procedure/diagnosis of the same Patient/Hospitalization pair is "True", the final dataframe will be "True". The basic assumption is that if out of all the procedures/diagnoses one is a potentially planned one, the whole hospitalization was planned.

In [63]:
def unstack_planned(df, step):
    """
    Function to get the unique values for each subject/hospitalization
    """
    unstacked = (df.groupby(['subject_id', 'hadm_id'])[step]
        .apply(lambda x: pd.Series(x.max()))
        .unstack())
    unstacked.rename(columns = {0:step}, inplace = True)
    unstacked.reset_index(inplace = True)
    return unstacked

In [64]:
unstacked_step_1 = unstack_planned(proc_icd, "Step 1")
unstacked_step_2 = unstack_planned(diag_icd, "Step 2")
unstacked_step_3 = unstack_planned(proc_icd, "Step 3 - def")
unstacked_step_4 = unstack_planned(diag_icd, "Step 4 - def")

In [65]:
unstacked_step_2.shape

(58976, 3)

In [66]:
unstacked_step_1.shape

(52243, 3)

In [67]:
df_clean = pd.merge(df_clean, unstacked_step_1, how = 'left', on=['subject_id', 'hadm_id'])
df_clean = pd.merge(df_clean, unstacked_step_2, how = 'left', on=['subject_id', 'hadm_id'])
df_clean = pd.merge(df_clean, unstacked_step_3, how = 'left', on=['subject_id', 'hadm_id'])
df_clean = pd.merge(df_clean, unstacked_step_4, how = 'left', on=['subject_id', 'hadm_id'])

There are unfortunately missing data in procedures for certain pairs of Patients/Hospitalizations - with consequently missing data on Step 3 and Step 1 of our Planned Readmission Algorithm. We will consider them as unplanned procedures while processing our algorithm.

In [68]:
df_clean.isnull().sum()

subject_id           0
hadm_id              0
admittime            0
dischtime            0
first_careunit       0
last_careunit        0
age                  0
gender               0
marital_status    1633
insurance            0
diagnosis            1
text                 0
Step 1            5298
Step 2               0
Step 3 - def      5298
Step 4 - def         0
dtype: int64

## Readmissions

We now move onto calculating the **readmissions**.

In [69]:
# sort by subject, admission and admission time
df_clean = df_clean.sort_values(by=["subject_id", "admittime", "hadm_id"])
df_clean.reset_index(inplace = True)

In [70]:
df_clean.head(10)

,index,subject_id,hadm_id,admittime,dischtime,first_careunit,last_careunit,age,gender,marital_status,insurance,diagnosis,text,Step 1,Step 2,Step 3 - def,Step 4 - def
0,0,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,MICU,MICU,76.526788,M,MARRIED,Medicare,HYPOTENSION,Admission Date: [**2101-10-20**] Discharg...,False,False,False,True
1,1,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,MICU,MICU,47.845044,F,SINGLE,Private,"FEVER,DEHYDRATION,FAILURE TO THRIVE",Admission Date: [**2191-3-16**] Discharge...,False,False,False,True
2,2,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,SICU,SICU,65.940670,F,MARRIED,Medicare,CHRONIC RENAL FAILURE/SDA,Admission Date: [**2175-5-30**] Dischar...,True,False,False,True
3,3,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,SICU,SICU,50.148292,F,MARRIED,Private,BRAIN MASS,Admission Date: [**2178-4-16**] ...,False,False,False,False
4,4,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,CCU,CSRU,39.866116,F,None,Medicaid,CORONARY ARTERY DISEASE,Admission Date: [**2167-1-8**] Discharg...,False,False,True,True
5,6,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,CSRU,CSRU,47.454296,F,MARRIED,Private,PATIENT FORAMEN OVALE\ PATENT FORAMEN OVALE MI...,Admission Date: [**2134-12-27**] ...,False,False,True,True
6,5,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,CSRU,CSRU,47.819229,F,MARRIED,Private,PERICARDIAL EFFUSION,Admission Date: [**2135-5-9**] D...,False,False,True,True
7,7,18,188822,2167-10-02 11:18:00,2167-10-04 16:15:00,CCU,CCU,50.841554,M,MARRIED,Private,HYPOGLYCEMIA;SEIZURES,Admission Date: [**2167-10-2**] ...,False,False,False,True
8,8,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,TSICU,TSICU,300.002968,M,None,Medicare,C-2 FRACTURE,Admission Date: [**2108-8-5**] D...,NaN,False,NaN,True
9,9,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,CSRU,CSRU,75.876833,F,WIDOWED,Medicare,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,Admission Date: [**2183-4-28**] Dischar...,False,False,True,True


In [71]:
# calculate time delta between subsequent readmissions of the same patient 
df_clean['readmit_dt'] = np.zeros(df_clean.shape[0])
df_clean['next_readmit_dt'] = np.zeros(df_clean.shape[0])
df_clean['readmit_last_careunit'] = None
df_clean['adm_num'] = np.zeros(df_clean.shape[0])
df_clean['step_1_readmit'] = None
df_clean['step_2_readmit'] = None
df_clean['step_3_readmit'] = None
df_clean['step_4_readmit'] = None

for idx in np.arange(1,df_clean.shape[0]):
    if df_clean.subject_id[idx] == df_clean.subject_id[idx - 1]:     
        prev_disch = df_clean.dischtime[idx-1]
        curr_adm = df_clean.admittime[idx]
        dt = curr_adm - prev_disch
        dt_hrs_calc = np.round(dt.value/3600.0/1e9,2)

        df_clean.loc[idx,'adm_num'] = df_clean['adm_num'][idx-1] + 1
        df_clean.loc[idx, 'readmit_dt'] = dt_hrs_calc
        df_clean.loc[idx-1, 'next_readmit_dt'] = dt_hrs_calc
        df_clean.loc[idx, 'readmit_last_careunit'] = df_clean['last_careunit'][idx-1]
        
        # we need to check planned/unplanned readmission too
        df_clean.loc[idx-1, 'step_1_readmit'] = df_clean['Step 1'][idx]
        df_clean.loc[idx-1, 'step_2_readmit'] = df_clean['Step 2'][idx]
        df_clean.loc[idx-1, 'step_3_readmit'] = df_clean['Step 3 - def'][idx]
        df_clean.loc[idx-1, 'step_4_readmit'] = df_clean['Step 4 - def'][idx]

And then we calculate which of these readmissions happen before 30 days.

In [72]:
# Define threshold in hours
threshold = 30*24

In [73]:
# Define label column based on threshold
df_clean['future_readmit'] = None
df_clean['future_readmit'] = ['No' if dt == 0.0 else 'Yes' if dt<=threshold else 'No' for dt in df_clean.next_readmit_dt]

In [74]:
df_clean.head(50)

,index,subject_id,hadm_id,admittime,dischtime,first_careunit,last_careunit,age,gender,marital_status,...,Step 4 - def,readmit_dt,next_readmit_dt,readmit_last_careunit,adm_num,step_1_readmit,step_2_readmit,step_3_readmit,step_4_readmit,future_readmit
0,0,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,MICU,MICU,76.526788,M,MARRIED,...,True,0.00,0.00,None,0.0,None,None,None,None,No
1,1,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,MICU,MICU,47.845044,F,SINGLE,...,True,0.00,0.00,None,0.0,None,None,None,None,No
2,2,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,SICU,SICU,65.940670,F,MARRIED,...,True,0.00,0.00,None,0.0,None,None,None,None,No
3,3,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,SICU,SICU,50.148292,F,MARRIED,...,False,0.00,0.00,None,0.0,None,None,None,None,No
4,4,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,CCU,CSRU,39.866116,F,None,...,True,0.00,0.00,None,0.0,None,None,None,None,No
5,6,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,CSRU,CSRU,47.454296,F,MARRIED,...,True,0.00,3094.10,None,0.0,False,False,True,True,No
6,5,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,CSRU,CSRU,47.819229,F,MARRIED,...,True,3094.10,0.00,CSRU,1.0,None,None,None,None,No
7,7,18,188822,2167-10-02 11:18:00,2167-10-04 16:15:00,CCU,CCU,50.841554,M,MARRIED,...,True,0.00,0.00,None,0.0,None,None,None,None,No
8,8,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,TSICU,TSICU,300.002968,M,None,...,True,0.00,0.00,None,0.0,None,None,None,None,No
9,9,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,CSRU,CSRU,75.876833,F,WIDOWED,...,True,0.00,0.00,None,0.0,None,None,None,None,No


In [75]:
print ('Value counts:')
print (df_clean.future_readmit.value_counts())
print ('\nValue proportions:')
print (df_clean.future_readmit.value_counts()/df_clean.shape[0])
print ('\nReadmission counts:')
print (df_clean.adm_num.value_counts())

Value counts:
No     40381
Yes     2444
Name: future_readmit, dtype: int64

Value proportions:
No     0.942931
Yes    0.057069
Name: future_readmit, dtype: float64

Readmission counts:
0.0     33348
1.0      5650
2.0      1798
3.0       810
4.0       416
5.0       217
6.0       143
7.0       100
8.0        68
9.0        49
10.0       36
11.0       24
12.0       21
13.0       18
14.0       15
15.0       15
16.0       12
17.0       10
18.0        9
19.0        8
20.0        7
21.0        5
23.0        5
22.0        5
24.0        4
25.0        4
26.0        4
27.0        4
29.0        3
30.0        3
28.0        3
31.0        2
32.0        2
33.0        1
34.0        1
35.0        1
36.0        1
37.0        1
38.0        1
39.0        1
Name: adm_num, dtype: int64


There is only one case of negative readmission time, however it is mostly due to a mistake in the database. As we can read from the discharge notes, it was discharged and readmitted on the same date.

In [76]:
df_clean[(df_clean.readmit_dt < 0) | (df_clean.next_readmit_dt < 0)]

,index,subject_id,hadm_id,admittime,dischtime,first_careunit,last_careunit,age,gender,marital_status,...,Step 4 - def,readmit_dt,next_readmit_dt,readmit_last_careunit,adm_num,step_1_readmit,step_2_readmit,step_3_readmit,step_4_readmit,future_readmit
4131,4131,5246,119872,2185-11-14 08:36:00,2185-12-21 18:15:00,MICU,MICU,74.075157,M,SINGLE,...,True,0.00,-450.37,None,0.0,False,False,False,True,Yes
4132,4132,5246,199031,2185-12-02 23:53:00,2185-12-21 18:15:00,MICU,MICU,74.126182,M,SINGLE,...,True,-450.37,0.00,MICU,1.0,None,None,None,None,No


In [77]:
df_clean.text[4131]

"Admission Date:  [**2185-11-14**]              Discharge Date:   [**2185-12-2**]\n\nDate of Birth:  [**2111-10-19**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 1973**]\nChief Complaint:\nUnresponsiveness, Hct 14.7\n\nMajor Surgical or Invasive Procedure:\nRight femoral line\nIntubation/mechanical ventilation\nEndoscopy\nColonscopy\n\n\nHistory of Present Illness:\nPt is 74 yo m w/ COPD, DM, h/o prostate CA, who presented to the\nED after having decreased responsiveness at his nursing homee.\nPt had FS to 300's, received insulin, and FS decreased to 180's\nafter EMS arrived.  Pt reportedly admitted to SOB x several\nweeks, but denies CP.  No N/V/F/C.  Denies hematemesis or bloody\nstools.  Pt had increasing edema over past several weeks, and\nLasix was recently increased from 40mg qd to 40mg [**Hospital1 **].\n.\nIn the [**Name (NI) **], pt was found to have hct of 14.8 and melena on r

In [78]:
df_clean.text[4132]

"Admission Date:  [**2185-12-2**]              Discharge Date:   [**2185-12-21**]\n\nDate of Birth:  [**2111-10-19**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 1974**]\nChief Complaint:\n[**First Name3 (LF) **], MS change\n\nMajor Surgical or Invasive Procedure:\nnone\n\nHistory of Present Illness:\n74M discharged from [**Hospital1 18**] today, p/w MS change, [**Hospital1 **] to\n70-85% RA.  Pt was just admitted for blood loss anemia secondary\nto GI bleed.  Hospital course was complicated by respiratory\nfailure secondary to VAP and CHF, thrombocytopenia, MS change.\nHe was discharged to NH home, and then developed MS change and\n[**Hospital1 **].  There are no records from the NH with regards to\ndetails of MS [**First Name (Titles) **] [**Last Name (Titles) **].\n.\nIn ED, he had improvement in MS [**First Name (Titles) 151**] [**Last Name (Titles) 20358**].  Then, he became\nagita

We have now three types of data:

1. First hospitalization
2. Second hospitalization - Readmission within 30 days
3. Second hospitalization - Readmission after 30 days

Since we need to predict the second readmission, we need to drop all the second hospitalization, **even those which happened after 30 days, as they will indeed represent patiens who are suffering of some recurring problems** (*and thus we cannot really use such observations as First hospitalizations*).

We will also drop from Third Hospitalization on for the same reason, even if the second/third/etc. hospitalization is a completely separate event (*i.e. happening after 30 days*).

In [79]:
if expanded_def:
    df_final = df_clean
else:
    df_final = df_clean.loc[df_clean['adm_num'] == 0,:].copy()

In [80]:
df_final.head(10)

,index,subject_id,hadm_id,admittime,dischtime,first_careunit,last_careunit,age,gender,marital_status,...,Step 4 - def,readmit_dt,next_readmit_dt,readmit_last_careunit,adm_num,step_1_readmit,step_2_readmit,step_3_readmit,step_4_readmit,future_readmit
0,0,3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,MICU,MICU,76.526788,M,MARRIED,...,True,0.0,0.0,None,0.0,None,None,None,None,No
1,1,4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,MICU,MICU,47.845044,F,SINGLE,...,True,0.0,0.0,None,0.0,None,None,None,None,No
2,2,6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,SICU,SICU,65.940670,F,MARRIED,...,True,0.0,0.0,None,0.0,None,None,None,None,No
3,3,11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,SICU,SICU,50.148292,F,MARRIED,...,False,0.0,0.0,None,0.0,None,None,None,None,No
4,4,13,143045,2167-01-08 18:43:00,2167-01-15 15:15:00,CCU,CSRU,39.866116,F,None,...,True,0.0,0.0,None,0.0,None,None,None,None,No
5,6,17,194023,2134-12-27 07:15:00,2134-12-31 16:05:00,CSRU,CSRU,47.454296,F,MARRIED,...,True,0.0,3094.1,None,0.0,False,False,True,True,No
6,5,17,161087,2135-05-09 14:11:00,2135-05-13 14:40:00,CSRU,CSRU,47.819229,F,MARRIED,...,True,3094.1,0.0,CSRU,1.0,None,None,None,None,No
7,7,18,188822,2167-10-02 11:18:00,2167-10-04 16:15:00,CCU,CCU,50.841554,M,MARRIED,...,True,0.0,0.0,None,0.0,None,None,None,None,No
8,8,19,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,TSICU,TSICU,300.002968,M,None,...,True,0.0,0.0,None,0.0,None,None,None,None,No
9,9,20,157681,2183-04-28 09:45:00,2183-05-03 14:45:00,CSRU,CSRU,75.876833,F,WIDOWED,...,True,0.0,0.0,None,0.0,None,None,None,None,No


In [81]:
print ('Value counts:')
print (df_final.future_readmit.value_counts())
print ('\nValue proportions:')
print (df_final.future_readmit.value_counts()/df_final.shape[0])
print ('\nReadmission counts:')
print (df_final.adm_num.value_counts())

Value counts:
No     40381
Yes     2444
Name: future_readmit, dtype: int64

Value proportions:
No     0.942931
Yes    0.057069
Name: future_readmit, dtype: float64

Readmission counts:
0.0     33348
1.0      5650
2.0      1798
3.0       810
4.0       416
5.0       217
6.0       143
7.0       100
8.0        68
9.0        49
10.0       36
11.0       24
12.0       21
13.0       18
14.0       15
15.0       15
16.0       12
17.0       10
18.0        9
19.0        8
20.0        7
21.0        5
23.0        5
22.0        5
24.0        4
25.0        4
26.0        4
27.0        4
29.0        3
30.0        3
28.0        3
31.0        2
32.0        2
33.0        1
34.0        1
35.0        1
36.0        1
37.0        1
38.0        1
39.0        1
Name: adm_num, dtype: int64


In [82]:
# we drop columns we will not be using anymore
df_final.drop(['readmit_dt', 'readmit_last_careunit', 'adm_num', 'Step 1', 'Step 2', 'Step 3 - def', 'Step 4 - def'], axis = 1, inplace = True)
df_final.reset_index(inplace = True, drop = True)

We now need to distinguish between planned and unplanned readmissions using the aformentioned algorithm.

In [83]:
def planned_admission(row):
    """
    Function to apply the Planned Readmission Algorithm
    """
    if row["step_1_readmit"] == True:
        row['planned_readmit'] = 'Planned'
    elif row["step_2_readmit"] == True:
        row['planned_readmit'] = 'Planned'
    elif row["step_3_readmit"] == True and row["step_4_readmit"] == False:
        row['planned_readmit'] = 'Planned'
    else:
        row['planned_readmit'] = 'Unplanned'
    return row['planned_readmit']

In [84]:
df_final['planned_readmit'] = df_final.apply(planned_admission, axis = 1)

In [85]:
if expanded_def == False:
    # replace with "No" for all hospitalizations without readmission
    df_final['planned_readmit'] = np.where(df_final['future_readmit'] == 'No', 'No', df_final['planned_readmit'])

In [86]:
df_final.planned_readmit.value_counts()

Unplanned    42629
Planned        196
Name: planned_readmit, dtype: int64

In [87]:
df_final.step_1_readmit.value_counts()

False    7887
True      105
Name: step_1_readmit, dtype: int64

In [88]:
df_final.step_2_readmit.value_counts()

False    9461
True       16
Name: step_2_readmit, dtype: int64

In [89]:
df_final.step_3_readmit.value_counts()

False    5598
True     2394
Name: step_3_readmit, dtype: int64

In [90]:
df_final.step_4_readmit.value_counts()

True     9311
False     166
Name: step_4_readmit, dtype: int64

As we can see planned readmissions are not many. In any case, they will be **dropped**.

In [93]:
# drop readmissions
if expanded_def == False:
    df_final_v02 = df_final[df_final['planned_readmit'] != 'Planned']
else:
    df_final_v02 = df_final.copy()

In [96]:
# define our target
if expanded_def:
    df_final_v02['target'] = np.where(df_final_v02['future_readmit'] == 'Yes', 1, 0)
else:
    df_final_v02['target'] = np.where(df_final_v02['planned_readmit'] == 'Unplanned', 1, 0)

In [97]:
df_final_v02.target.value_counts()

0    40381
1     2444
Name: target, dtype: int64

In [98]:
df_final_v02.shape

(42825, 21)

In [99]:
df_final_v02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42825 entries, 0 to 42824
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            42825 non-null  int64         
 1   subject_id       42825 non-null  int64         
 2   hadm_id          42825 non-null  int64         
 3   admittime        42825 non-null  datetime64[ns]
 4   dischtime        42825 non-null  datetime64[ns]
 5   first_careunit   42825 non-null  object        
 6   last_careunit    42825 non-null  object        
 7   age              42825 non-null  float64       
 8   gender           42825 non-null  object        
 9   marital_status   41192 non-null  object        
 10  insurance        42825 non-null  object        
 11  diagnosis        42824 non-null  object        
 12  text             42825 non-null  object        
 13  next_readmit_dt  42825 non-null  float64       
 14  step_1_readmit   7992 non-null   objec

In [100]:
# and save our cleaned dataframe for next usage
# drop columns we do not need anymore
df_final_v02.drop(['step_1_readmit', 'step_2_readmit', 'step_3_readmit', 'step_4_readmit','future_readmit','planned_readmit'], axis = 1, inplace = True)
# need to reset the index
df_final_v02.reset_index(inplace=True, drop = True)
# save our dataset up to now in feather format
df_final_v02.to_feather(f'{path_to_data}df_cleaned')